In [2]:
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoderfrom sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score,classification_report


In [3]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')
df.bmi.replace(to_replace=np.nan, value=df.bmi.mean(),inplace=True)
x = df.iloc[:,1:-1].values
y = df.iloc[:,-1].values
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0,5,9])],remainder='passthrough')
x = np.array(ct.fit_transform(x))
le = LabelEncoder()
x[:, 15] = le.fit_transform(x[:, 15])
x[:, 16] = le.fit_transform(x[:, 16])
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train,y_train.ravel())

classifier = XGBClassifier(eval_metric= 'error', learning_rate= 0.1)
classifier.fit(X_train_res, y_train_res)
y_pred = classifier.predict(X_test)
y_prob = classifier.predict_proba(X_test)[:,1]
cm = confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))
print(f'ROC AUC score: {roc_auc_score(y_test, y_prob)}')
print('Accuracy Score: ',accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       968
           1       0.57      0.07      0.13        54

    accuracy                           0.95      1022
   macro avg       0.76      0.54      0.55      1022
weighted avg       0.93      0.95      0.93      1022

ROC AUC score: 0.785563973063973
Accuracy Score:  0.9481409001956947
